### Tests for the 2nd Research question (Culture)

In [ ]:
import os
from nltk import tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re


##############################################################################################################


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")


##############################################################################################################


listStructureWords =  []

dfSrtucturingWords = pd.read_excel("../structureWords.xlsx")

listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []

df_fileName = pd.DataFrame({})

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\only100-German-teachers"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\only100-German-teachers", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)

##############################################################################################################

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

##############################################################################################################


k = 19

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

df_clusters.to_excel("GermanExcelResearchQ1.xlsx")

##############################################################################################################


df_test = pd.DataFrame({})

df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\Teachers prompt\Goldstandards.xlsx")

def_clusters = pd.read_excel(r"GermanExcelResearchQ1.xlsx")

listRef = df_ref['Gold standard']

listClusters = def_clusters['cluster']
listNewSentences = def_clusters['text']
listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

cosListRefToClusters = []
mm=0
for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": index_max
              },ignore_index=True)
    mm = mm+1


df_test.to_excel("GermanExcelResearchQ1.xlsx");


##############################################################################################################

tpCount= 0
tnCount  = 0

fnCount = 0
fpcount = 0


dfPairEvaluation = pd.read_excel(r"GermanExcelResearchQ1.xlsx")

iteration = dfPairEvaluation.count()['ID']

for i in range(0,iteration):
    for j in range (i+1, iteration):


        clusterSystem1 = dfPairEvaluation.query(f"ID == {i}")['cluster'].iloc[0]
        clusterGold1 = dfPairEvaluation.query(f"ID == {i}")['cos Sim'].iloc[0]
        clusterSystem2 = dfPairEvaluation.query(f"ID == {j}")['cluster'].iloc[0]
        clusterGold2 = dfPairEvaluation.query(f"ID == {j}")['cos Sim'].iloc[0]


        if (clusterSystem1 == clusterSystem2):

            if (clusterGold1 ==clusterGold2): # it is a true positive
                tpCount = tpCount+ 1

            else:
                fnCount = fnCount + 1   # it is a false negative
        else:
            if (clusterGold1 !=clusterGold2):
                tnCount = tnCount + 1  # it is a true negative
            else:
                fpcount = fpcount+1   # it is a false positive.

print("the value of tp is", tpCount)
print("the value of tn is", tnCount)
print("the value of fn is", fnCount)
print("the value of fb is", fpcount)

accuracy = (tpCount + tnCount)/(tpCount +tnCount + fpcount+ fnCount)

print("The accuracy is ",accuracy*100)  # the value is higher than the other accuracy.

percision = tpCount / (tpCount + fpcount)

print("The percision is ",percision*100)

recall = tpCount / (tpCount + fnCount)

print("The recall is ",recall*100)

##############################################################################################################

from collections import defaultdict
dfPurity = pd.read_excel(r"GermanExcelResearchQ1.xlsx")
d = defaultdict(list)

for j in range(0, k-1):
    mylist = dfPurity.query(f"cluster == {j}")['cos Sim']
    d[j].extend(mylist)

rightCounter = 0

for key, value in d.items():
    mostFreq = max(set(value), key = value.count)
    rightCounter = rightCounter + value.count(mostFreq)
    #print(key , value.count(mostFreq))

total = dfPurity.count()['ID']

print("purity = ", (rightCounter/total)*100)

In [ ]:
import os
from nltk import tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re


##############################################################################################################


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")


##############################################################################################################


listStructureWords =  []

dfSrtucturingWords = pd.read_excel("../structureWords.xlsx")

listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []

df_fileName = pd.DataFrame({})

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100 - German Ads"):
   if (len(sentences) >= 1000):
        break
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100 - German Ads", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                    sentences.append(ss)
##############################################################################################################

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

##############################################################################################################


k = 19

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

df_clusters.to_excel("GermanExcelResearchQ1-Normalised-AD.xlsx")

##############################################################################################################


df_test = pd.DataFrame({})

df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\Teachers prompt\Goldstandards.xlsx")

def_clusters = pd.read_excel(r"GermanExcelResearchQ1-Normalised-AD.xlsx")

listRef = df_ref['Gold standard']

listClusters = def_clusters['cluster']
listNewSentences = def_clusters['text']
listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

cosListRefToClusters = []
mm=0
for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": index_max
              },ignore_index=True)
    mm = mm+1

df_test.to_excel("GermanExcelResearchQ1-Normalised-AD.xlsx");



### Test for the cosin Similarity

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

#Sentences are encoded by calling model.encode()
emb1 = model.encode("This is a red cat with a hat.")
emb2 = model.encode("My cat is red and has a hat")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

### final results cluster

In [ ]:
from pathlib import Path
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
import gensim
from nltk.corpus import stopwords
import sentence_transformers
from sentence_transformers import SentenceTransformer,util
from sklearn.cluster import KMeans
import re
import statistics
from keras.layers import Average


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

listStructureWords =  []
dfSrtucturingWords = pd.read_excel("../structureWords.xlsx")
listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()
regex = r"\b(?:{})\b".format("|".join(listStructureWords))

sentences = []
fileNames= []
j=0


for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\10 essays for essays reference - ads\1"):
   my_file = Path(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\10 essays for essays reference - ads\1", filename)
   if my_file.is_file():
       text = my_file.read_text(encoding='utf-8-sig')
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)
                     fileNames.append(filename)

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")


def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

k = 19
clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)


df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\Ads prompt\01.08.22\new Gold standards.xlsx")

listRef = df_ref['Gold standard']

listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

listIndexMax =[]

for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    listIndexMax.append(index_max)

    df_fileName = df_fileName.append({
        "FinalCluster": index_max
              },ignore_index=True)

df_final = pd.DataFrame({
    "sentence": sentences,
    "fileName": fileNames,
    "cluster": cluster_labels,
    "FinalCluster": listIndexMax
})

df_final.to_excel("FinalAdResults.xlsx");
